<a href="https://colab.research.google.com/github/cortiz313/Machine-Learning-Class/blob/main/projects/Image_Classification_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Classification Project
By: Christian Ortiz and Sejal Nathu-Hari

Note: Just as we did for Project 2, we collaborated over Zoom for the entirety of the project, working through the assignment together. There was no separation in the work, we added to it and worked on it all together

## 1) Reading and Observing the Data

Below we will read the CIFAR10 dataset into variables, and take a look at the shape and description of the data

In [6]:
# Importing Keras library from TensorFlow
from tensorflow import keras

# Importing train_test_split to split the dataset into training and testing subsets, or training and validation sets
from sklearn.model_selection import train_test_split

In [ ]:
# Load the CIFAR-10 dataset and assign to variables
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

In [3]:
# Print the shape of each variable
print("Shape of train_images:", train_images.shape)
print("Shape of train_labels:", train_labels.shape)
print("Shape of test_images:", test_images.shape)
print("Shape of test_labels:", test_labels.shape)

Shape of train_images: (50000, 32, 32, 3)
Shape of train_labels: (50000, 1)
Shape of test_images: (10000, 32, 32, 3)
Shape of test_labels: (10000, 1)


We see that the training data has 50,000 images of shape 32,32,3 and the test data has 10,000 images of size 32,32,3 as well. The labels, of course, have the same number of images, with only a single column corresponding to the label of the image.

## 2) Splitting and Manipulating the Data

Now we will split the training set into training and validation sets in order to fit our model later

In [7]:
# Split the training data into training and validation sets. 80% in the training sets and 20% in the validation sets
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42)

Below we look at the shapes of the image sets to observe the split

In [10]:
train_images.shape

(40000, 32, 32, 3)

In [11]:
val_images.shape

(10000, 32, 32, 3)

As expected, we find 40,000 images in the train set and 10,000 in the validation set.

Now we will normalize the pixel values, so instead of being from 0 to 255, they are from 0 to 1

In [12]:
# Normalize pixel values to be between 0 and 1 for training and testing images
train_images, test_images = train_images / 255., test_images / 255.

In [13]:
# Normalize validation images
val_images = val_images / 255.

Just to check the values briefly to make sure they are normalized, let's look at the values of the first image in this train_images array

In [ ]:
train_images[0]

## 3) Creating our CNN

Below we will define the architecture of the model, adding several layers that will help us to classify the CIFAR-10 images

In [35]:
# Define the model architecture
model = keras.Sequential([
    # Add a convolutional layer with 32 filters, each of size 3x3 with ReLU activation
    # Set the input shape to be 32x32 pixels with 3 color channels, because that is the shape of the CIFAR-10 images as we saw earlier
    keras.layers.Conv2D(32, kernel_size=(3,3) ,  activation="relu", input_shape=(32, 32, 3)),
    
    # Add a max pooling layer with a pool size of 2x2
    keras.layers.MaxPooling2D((2, 2)),
    
    # Add another convolutional layer with 64 filters, each of size 3x3 with ReLU activation
    keras.layers.Conv2D(64, kernel_size=(3,3), activation="relu"),
    
    # Add another max pooling layer with a pool size of 2x2
    keras.layers.MaxPooling2D((2, 2)),
    
    # Flatten the output from the previous layer
    keras.layers.Flatten(),
    
    # Add a fully connected dense layer with 64 units and ReLU activation
    keras.layers.Dense(64, activation="relu"),
    
    # Add a fully connected dense layer with 10 units (one for each class)
    # We tried adding the activation parameter here, but it led to worst results in accuracy
    keras.layers.Dense(10)
])

In [23]:
# this line displays the layers in the model and the number of parameters in each layer
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 2304)              0         
                                                                 
 dense_4 (Dense)             (None, 64)               

## 4) Compiling the Model

So initially, we were compiling the model as seen below, but we were getting very low accuracy on both the validation data and the test data. So after some research, we found that the SparseCategoricalCrossentropy would work better for our data. Our classification problem in this project involves multi-class classification, i.e. classifying the images into one of ten possible categories, so it makes sense to use a loss function that is specifically designed for multi-class classification tasks, which Sparse Categorical Crossentropy is made for.
Binary crossentropy is used when there are only two possible classes to classify. 

<br>

The choice of using the "adam" optimizer was because we found that it is a more advanced optimization algortihm, and combines the benefits of two other popular algos, one being RMSProp which we use here, and the other being AdaGrad. We tested it out, and found that we had a higher accuracy on both the validation and the test data, so we kept it.

In [ ]:
# model.compile(optimizer=optimizers.RMSprop(lr=1e-4),
#                 loss='binary_crossentropy',
#                 metrics=['accuracy'])

In [36]:
# Compile the model with the adam optimizer
model.compile(optimizer="adam",
              # Sparse categorical crossentropy is used as the loss function, since we are dealing with a multiclass classification problem.
              # We set from_logits=True because our last layer does not have a softmax activation function
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              # We use accuracy as the metric to monitor during training
              metrics=['accuracy'])

## 5) Fitting the Model

Here we fit the model. We tried a different number of epochs (5,10,20), and different batch sizes (32,256), and we found 10 epochs and a batch size of 256 gave us the best result. 

In [39]:
# Train the model on the training set and validate it on the validation set for 20 epochs, using a batch size of 32.
hist = model.fit(train_images, train_labels, validation_data=(val_images, val_labels), epochs=10, batch_size=256)

Epoch 1/10
157/157 [==============================] - 2s 11ms/step - loss: 0.7549 - accuracy: 0.7388 - val_loss: 0.9096 - val_accuracy: 0.6854
Epoch 2/10
157/157 [==============================] - 1s 8ms/step - loss: 0.7242 - accuracy: 0.7499 - val_loss: 0.8988 - val_accuracy: 0.6872
Epoch 3/10
157/157 [==============================] - 1s 8ms/step - loss: 0.7107 - accuracy: 0.7560 - val_loss: 0.8944 - val_accuracy: 0.6912
Epoch 4/10
157/157 [==============================] - 1s 8ms/step - loss: 0.6981 - accuracy: 0.7585 - val_loss: 0.8855 - val_accuracy: 0.6935
Epoch 5/10
157/157 [==============================] - 1s 8ms/step - loss: 0.6853 - accuracy: 0.7638 - val_loss: 0.9159 - val_accuracy: 0.6858
Epoch 6/10
157/157 [==============================] - 1s 9ms/step - loss: 0.6737 - accuracy: 0.7684 - val_loss: 0.8876 - val_accuracy: 0.6971
Epoch 7/10
157/157 [==============================] - 2s 10ms/step - loss: 0.6580 - accuracy: 0.7733 - val_loss: 0.8826 - val_accuracy: 0.6998
Epoc

## 6) Printing the Accuracy on the Test Data

In [42]:
# Get the model's accuracy on the test set
acc_score = model.evaluate(test_images, test_labels)

# Print the accuracy score
print("Accuracy: ", acc_score[1])

313/313 [==============================] - 1s 3ms/step - loss: 0.9144 - accuracy: 0.6956
Accuracy:  0.6955999732017517


We were able to achieve a 69.6% accuracy score on our test data, which was the best score we were able to get from our tweaks. On other tests, we hovered around 65-67%.

<br>

Overall, although we know with more advanced methods you can probably get a higher score, we are happy with the score we were able to achieve. 